# Calcualte performance rankings from cleaned results

In [1]:
# read cleaned results
import pandas as pd
from pathlib import Path
import pdb, os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

agg_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results_with_default.csv"))
agg_df = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results.csv"))

fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))
fold_df = pd.read_csv(Path("./cleaned_results/tuned_fold_results.csv"))

# make sure output folder exists
output_folder = Path("./performance_rankings")
output_folder.mkdir(exist_ok=True)  

In [2]:
import numpy as np

def get_rank_table(df, metric, table_idx=4):
    """
    NOTE: the df needs to have the following columns defined:
    - {metric}_rank_mean
    - normalized_{metric}__test_mean
    """
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    df = df[df["dataset_name"].isin(DATASETS)]

    overall_ranks = df.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "median", "count"],
            f"normalized_{metric}__test_mean": ["mean", "median"],
            f"normalized_{metric}__test_std": ["mean", "median"],
            f"train_per_1000_inst_mean_{metric}": ["mean", "median"],        
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    # overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].apply(lambda x: "{:d}".format(int(x)))

    # mean/median mean-rank
    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "median")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "median")].apply(lambda x: "{:d}".format(int(x)) if int(x) == x else "{:.1f}".format(x))
    
    # normalized metric - mean and std over folds
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "median")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "median")].apply(lambda x: "{:.2f}".format(x))


    # normalized runtime
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "mean")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "median")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "median")].apply(lambda x: "{:.2f}".format(x))
   


    final_table = overall_ranks.set_index("alg_name")

    return final_table


### Save rank tables to file

In [5]:
# save rank tables to csv and latex
from analysis_utils import ALG_DISPLAY_NAMES
table_idx = 1

metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

final_tables = {}

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    agg_df_with_default

    # first with default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df_with_default, metric, table_idx = table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank_with_default.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank_with_default.tex", index=True, escape=False)

    # now without default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df, metric, table_idx=table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank.tex", index=True, escape=False)

/tmp/ipykernel_1424352/2689206630.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '3' '1' '1' '1' '9']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
/tmp/ipykernel_1424352/2689206630.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['38' '38' '35' '37' '40' '41' '41' '43' '42' '35' '41' '43' '42' '42'
 '42' '43' '44' '42' '39' '38' '41' '39' '44' '41' '44' '44' '42' '42'
 '42' '45' '43' '39' '45' '43' '43' '44' '44' '45' '45' '45' '43' '45'
 '43' '45' '45']' has dtype incompatible with 

In [7]:
final_tables['Accuracy']

Accuracy_rank_mean                    \
                              min max   mean median   
alg_name                                              
TabPFN                          1  21   5.45      4   
CatBoost                        1  20   5.84      4   
XGBoost                         1  21   7.51      6   
ResNet                          1  22   8.18      8   
SAINT                           1  22   8.51      7   
NODE                            1  22   8.53      8   
FTTransformer                   1  19   8.71    8.5   
RandomForest                    1  20   8.92      8   
LightGBM                        1  23   9.32      9   
TabFast                         1  23   9.71    9.5   
SVM                             1  21  10.01   10.5   
MLP-rtdl                        1  21  10.53     11   
DANet                           1  22  10.71     10   
DeepFM                          1  23  11.52     12   
DecisionTree                    1  22  12.87     14   
STG                             1  23  12.96     14   
MLP                             1  22  13.06     14   
LinearModel                     1  22  13.17     15   
TabTransformer                  1  23  14.00     15   
TabNet                          1  23  14.01     15   
KNN                             1  23  15.11     16   
VIME                            2  23  16.42     18   
NAM                             1  23  17.39     19   

               normalized_Accuracy__test_mean         \
                                         mean median   
alg_name                                               
TabPFN                                   0.85   0.93   
CatBoost                                 0.88   0.94   
XGBoost                                  0.83   0.92   
ResNet                                   0.78   0.86   
SAINT                                    0.76   0.87   
NODE                                     0.77   0.86   
FTTransformer                            0.79   0.84   
RandomForest                             0.80   0.86   
LightGBM                                 0.79   0.86   
TabFast                                  0.73   0.86   
SVM                                      0.72   0.83   
MLP-rtdl                                 0.69   0.77   
DANet                                    0.76   0.83   
DeepFM                                   0.67   0.77   
DecisionTree                             0.67   0.76   
STG                                      0.62   0.73   
MLP                                      0.62   0.63   
LinearModel                              0.57   0.63   
TabTransformer                           0.55   0.67   
TabNet                                   0.59   0.69   
KNN                                      0.53   0.60   
VIME                                     0.43   0.42   
NAM                                      0.39   0.35   

               normalized_Accuracy__test_std         \
                                        mean median   
alg_name                                              
TabPFN                                  0.32   0.24   
CatBoost                                0.26   0.16   
XGBoost                                 0.29   0.17   
ResNet                                  0.27   0.16   
SAINT                                   0.27   0.20   
NODE                                    0.23   0.15   
FTTransformer                           0.27   0.17   
RandomForest                            0.27   0.19   
LightGBM                                0.31   0.18   
TabFast                                 0.26   0.17   
SVM                                     0.23   0.16   
MLP-rtdl                                0.24   0.14   
DANet                                   0.28   0.18   
DeepFM                                  0.26   0.21   
DecisionTree                            0.31   0.20   
STG                                     0.25   0.14   
MLP                                     0.25   0.16   
LinearModel                